In [1]:
import json
import requests
from getpass import getpass
import sys
import time
import re
import threading
import datetime
import os
import pandas as pd
import geopandas as gpd
import pyproj
from shapely.geometry import box
from shapely import wkt



import warnings
warnings.filterwarnings("ignore")

/Users/hstouter/anaconda3/envs/remote_sensing/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


Connect to M2M serve

In [2]:
# Send HTTP request
def sendRequest(url, data, apiKey=None, exitIfNoResponse=True):
    """
    Send a request to an M2M (Machine-to-Machine) endpoint and return the parsed JSON response.

    Parameters:
    - url (str): The URL of the M2M endpoint.
    - data (dict): The payload to be sent with the request.
    - apiKey (str, optional): An optional API key for authorization. If not provided, the request will be sent without an authorization header.
    - exitIfNoResponse (bool, optional): If True, the program will exit upon receiving an error or no response. Defaults to True.

    Returns:
    - dict: The parsed JSON response containing the data, or False if there was an error.
    """
    
    # Convert payload to json string
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}              
        response = requests.post(url, json_data, headers = headers)  
    
    try:
      httpStatusCode = response.status_code 
      if response == None:
          print("No output from service")
          if exitIfNoResponse: sys.exit()
          else: return False
      output = json.loads(response.text)
      if output['errorCode'] != None:
          print(output['errorCode'], "- ", output['errorMessage'])
          if exitIfNoResponse: sys.exit()
          else: return False
      if  httpStatusCode == 404:
          print("404 Not Found")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 401: 
          print("401 Unauthorized")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 400:
          print("Error Code", httpStatusCode)
          if exitIfNoResponse: sys.exit()
          else: return False
    except Exception as e: 
          response.close()
          print(e)
          if exitIfNoResponse: sys.exit()
          else: return False
    response.close()
    
    return output['data']

In [3]:
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/" 

In [4]:
def downloadFile(url, out_dir):
    sema.acquire()
    try:
        response = requests.get(url, stream=True)
        disposition = response.headers['content-disposition']
        filename = re.findall("filename=(.+)", disposition)[0].strip("\"")
        print(f"    Downloading: {filename} -- {url}...")
        
        open(os.path.join(out_dir, filename), 'wb').write(response.content)
        sema.release()
    except Exception as e:
        print(f"\nFailed to download from {url}. Will try to re-download.")
        sema.release()
        runDownload(threads, url, out_dir)

In [5]:
maxthreads = 5 # Threads count for downloads
sema = threading.Semaphore(value=maxthreads)
label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") # Customized label using date time
threads = []

In [6]:
def runDownload(threads, url, out_dir):
    thread = threading.Thread(target=downloadFile, args=(url,out_dir,))
    threads.append(thread)
    thread.start()

In [7]:
outDir= '/Users/hstouter/Library/CloudStorage/Box-Box/NASA - Land Cover Land Use Change (2023 - 2026)/Data/LCLUC/RF_Datasets/landsat/LS8_9_test'

Connect to USGS 

Token:
H3xVeg!r!zyZoWtWqt2@uvIKIJVZ_ntvs89MIMLFMj6HRZWhewdPTor9C5iXQ@Ez

In [8]:
def prompt_ERS_login(serviceURL):
    print("Logging in...\n")

    p = ['Enter EROS Registration System (ERS) Username: ', 'Enter ERS Account Token: ']

    # Use requests.post() to make the login request
    response = requests.post(f"{serviceUrl}login-token", json={'username': getpass(prompt=p[0]), 'token': getpass(prompt=p[1])})

    if response.status_code == 200:  # Check for successful response
        apiKey = response.json()['data']
        print('\nLogin Successful, API Key Received!')
        headers = {'X-Auth-Token': apiKey}
        return apiKey
    else:
        print("\nLogin was unsuccessful, please try again or create an account at: https://ers.cr.usgs.gov/register.")

In [32]:
apiKey = prompt_ERS_login(serviceUrl) 

Logging in...


Login Successful, API Key Received!


Create json to submit request (select dataset, roi, time period, cloud coverage)

In [11]:
# to get correct CRS
pyproj.datadir.set_data_dir("/Users/hstouter/anaconda3/envs/remote_sensing/share/proj")

shp_path = ("/Users/hstouter/Desktop/NASA_LCLUC/RF_datasets/ynd_test_buffer.shp")
shp = gpd.read_file(shp_path)
print(shp.head())

# Check CRS & update if needed
print("original :", shp.crs)

# Update CRS
shp = shp.to_crs(epsg=4326)
print("updated :", shp.crs)

# Get bounding box coordinates
minx, miny, maxx, maxy = shp.total_bounds

# Create a shapely box (polygon)
bbox = box(minx, miny, maxx, maxy)

# Fix geometry (ensure it's valid and CCW)
bbox_fixed = bbox.buffer(0)


# Convert to GeoJSON-like dict
roi = bbox_fixed.__geo_interface__

# Wrap in API spatialFilter format
roi = {
    'filterType': 'geojson',
    'geoJson': roi
}

print(roi)


   Id                                           geometry
0   0  POLYGON ((1277436.254 393886.351, 1253203.36 3...
original : EPSG:3857
updated : EPSG:4326
{'filterType': 'geojson', 'geoJson': {'type': 'Polygon', 'coordinates': (((11.910780699769553, 3.5360943692198314), (11.257717326443535, 3.5360943692198314), (11.257717326443535, 4.1376852081059345), (11.910780699769553, 4.1376852081059345), (11.910780699769553, 3.5360943692198314)),)}}


## JSON INPUTS [EDIT]

In [40]:
datasetName = 'landsat_ot_c2_l2'
spatialFiler = roi
aquisitionFilter = {'start' : '2021-10-01', 'end' : '2022-09-30'}
cloudCoverFilter = {'min' : 0, 'max' : 80}


In [41]:
search_payload = {
    'datasetName' : datasetName,
    'sceneFilter' : {
        'spatialFilter' : spatialFiler,
        'acquisitionFilter' : aquisitionFilter,
        'cloudCoverFilter' : cloudCoverFilter
    }
}

search_payload

{'datasetName': 'landsat_ot_c2_l2',
 'sceneFilter': {'spatialFilter': {'filterType': 'geojson',
   'geoJson': {'type': 'Polygon',
    'coordinates': (((11.910780699769553, 3.5360943692198314),
      (11.257717326443535, 3.5360943692198314),
      (11.257717326443535, 4.1376852081059345),
      (11.910780699769553, 4.1376852081059345),
      (11.910780699769553, 3.5360943692198314)),)}},
  'acquisitionFilter': {'start': '2021-10-01', 'end': '2022-09-30'},
  'cloudCoverFilter': {'min': 0, 'max': 80}}}

In [13]:
scenes = sendRequest(serviceUrl + "scene-search", search_payload, apiKey) 


In [14]:
pd.json_normalize(scenes['results']) 


,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",26,LC81860572022262LGN00,LC08_L2SP_186057_20220919_20220928_02_T2,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2022-09-28 16:32:08-05,True,True,...,False,False,False,False,Polygon,"[[[9.45628, 3.28617], [9.45628, 5.3839], [11.5...",Polygon,"[[[9.45628, 3.64843], [11.13084, 3.28617], [11...",2022-09-19 00:00:00,2022-09-19 00:00:00
1,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",64,LC81850582022255LGN00,LC08_L2SP_185058_20220912_20220921_02_T1,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2022-09-21 18:56:40-05,True,True,...,False,False,False,False,Polygon,"[[[10.6995, 1.84011], [10.6995, 3.93761], [12....",Polygon,"[[[10.6995, 2.20203], [12.37109, 1.84011], [12...",2022-09-12 00:00:00,2022-09-12 00:00:00
2,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",77,LC91850582022247LGN01,LC09_L2SP_185058_20220904_20230330_02_T1,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2023-03-30 18:07:54-05,True,True,...,False,False,False,False,Polygon,"[[[10.71294, 1.84146], [10.71294, 3.93653], [1...",Polygon,"[[[10.71294, 2.2012], [12.38486, 1.84146], [12...",2022-09-04 00:00:00,2022-09-04 00:00:00
3,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",80,LC81850582022239LGN00,LC08_L2SP_185058_20220827_20220909_02_T2,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2022-09-09 16:11:03-05,True,True,...,False,False,False,False,Polygon,"[[[10.70631, 1.84026], [10.70631, 3.93772], [1...",Polygon,"[[[10.70631, 2.20221], [12.37774, 1.84026], [1...",2022-08-27 00:00:00,2022-08-27 00:00:00
4,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",67,LC91860572022238LGN01,LC09_L2SP_186057_20220826_20230401_02_T2,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2023-03-31 21:31:42-05,True,True,...,False,False,False,False,Polygon,"[[[9.46028, 3.28742], [9.46028, 5.38285], [11....",Polygon,"[[[9.46028, 3.64756], [11.13497, 3.28742], [11...",2022-08-26 00:00:00,2022-08-26 00:00:00
5,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",50,LC81860572022166LGN00,LC08_L2SP_186057_20220615_20220627_02_T1,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2022-06-27 18:31:38-05,True,True,...,False,False,False,False,Polygon,"[[[9.45935, 3.28627], [9.45935, 5.38414], [11....",Polygon,"[[[9.45935, 3.64871], [11.13376, 3.28627], [11...",2022-06-15 00:00:00,2022-06-15 00:00:00
6,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",55,LC91860572022158LGN01,LC09_L2SP_186057_20220607_20230414_02_T1,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2023-04-13 19:41:19-05,True,True,...,False,False,False,False,Polygon,"[[[9.47589, 3.28723], [9.47589, 5.38263], [11....",Polygon,"[[[9.47589, 3.64761], [11.15123, 3.28723], [11...",2022-06-07 00:00:00,2022-06-07 00:00:00
7,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",73,LC91850572022151LGN01,LC09_L2SP_185057_20220531_20230414_02_T2,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2023-04-14 15:15:23-05,True,True,...,False,False,False,False,Polygon,"[[[11.00811, 3.28759], [11.00811, 5.38305], [1...",Polygon,"[[[11.00811, 3.64795], [12.68322, 3.28759], [1...",2022-05-31 00:00:00,2022-05-31 00:00:00
8,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",71,LC81860572022150LGN00,LC08_L2SP_186057_20220530_20220603_02_T1,None,"[{'id': '5e83d1508031a4a3', 'fieldName': 'ID',...",None,2022-06-22 18:27:47-05,True,True,...,False,False,False,False,Polygon,"[[[9.48564, 3.28602], [9.48564, 5.38393], [11....",Polygon,"[[[9.48564, 3.64851], [11.16017, 3.28602], [11...",2022-05-30 00:00:00,2022-05-30 00:00:00
9,"[{'id': '5fb4ba12d7ec307f', 'brows

In [15]:
# Create scene ID list
{'listId': 'temp_landsat_ot_c2_l2_list',
     'idField': 'entityId',
     'entityIds': ['LC08_L2SP_068017_20200310_20200822_02_T1', 'LC08_L2SP_068018_20200310_20200822_02_T1'],
     'datasetName': 'landsat_ot_c2_l2'}

{'listId': 'temp_landsat_ot_c2_l2_list',
 'idField': 'entityId',
 'entityIds': ['LC08_L2SP_068017_20200310_20200822_02_T1',
  'LC08_L2SP_068018_20200310_20200822_02_T1'],
 'datasetName': 'landsat_ot_c2_l2'}

In [16]:
idField = 'entityId'

entityIds = []

for result in scenes['results']:
     # Add this scene to the list I would like to download if bulk is available
    if result['options']['bulk'] == True:
        entityIds.append(result[idField])
    
entityIds

['LC81860572022262LGN00',
 'LC81850582022255LGN00',
 'LC91850582022247LGN01',
 'LC81850582022239LGN00',
 'LC91860572022238LGN01',
 'LC81860572022166LGN00',
 'LC91860572022158LGN01',
 'LC91850572022151LGN01',
 'LC81860572022150LGN00',
 'LC81850582022143LGN00',
 'LC91850572022135LGN01',
 'LC91850582022135LGN01',
 'LC81860572022134LGN00',
 'LC91860572022126LGN01',
 'LC91850572022119LGN01',
 'LC81860572022118LGN00',
 'LC91860572022110LGN01',
 'LC91850572022103LGN01',
 'LC91850582022103LGN01',
 'LC81860572022102LGN00',
 'LC81850572022095LGN00',
 'LC81850582022095LGN00',
 'LC91850572022087LGN01',
 'LC81860572022086LGN00',
 'LC81850582022063LGN00',
 'LC91860572022062LGN01',
 'LC81860572022054LGN00',
 'LC81850572022047LGN00',
 'LC81850582022047LGN00',
 'LC91860572022046LGN01',
 'LC91850572022039LGN01',
 'LC91850582022039LGN01',
 'LC81860572022038LGN00',
 'LC81850572022031LGN00',
 'LC81850582022031LGN00',
 'LC91860572022030LGN01',
 'LC91850572022023LGN01',
 'LC91850582022023LGN01',
 'LC81860572

In [20]:
listId = "temp_{datasetName}_list" # customized list id
scn_list_add_payload = {
    "listId": listId,
    'idField' : idField,
    "entityIds": entityIds,
    "datasetName": "landsat_ot_c2_l2"
}
scn_list_add_payload

{'listId': 'temp_{datasetName}_list',
 'idField': 'entityId',
 'entityIds': ['LC81860572022262LGN00',
  'LC81850582022255LGN00',
  'LC91850582022247LGN01',
  'LC81850582022239LGN00',
  'LC91860572022238LGN01',
  'LC81860572022166LGN00',
  'LC91860572022158LGN01',
  'LC91850572022151LGN01',
  'LC81860572022150LGN00',
  'LC81850582022143LGN00',
  'LC91850572022135LGN01',
  'LC91850582022135LGN01',
  'LC81860572022134LGN00',
  'LC91860572022126LGN01',
  'LC91850572022119LGN01',
  'LC81860572022118LGN00',
  'LC91860572022110LGN01',
  'LC91850572022103LGN01',
  'LC91850582022103LGN01',
  'LC81860572022102LGN00',
  'LC81850572022095LGN00',
  'LC81850582022095LGN00',
  'LC91850572022087LGN01',
  'LC81860572022086LGN00',
  'LC81850582022063LGN00',
  'LC91860572022062LGN01',
  'LC81860572022054LGN00',
  'LC81850572022047LGN00',
  'LC81850582022047LGN00',
  'LC91860572022046LGN01',
  'LC91850572022039LGN01',
  'LC91850582022039LGN01',
  'LC81860572022038LGN00',
  'LC81850572022031LGN00',
  'LC81

In [21]:
count = sendRequest(serviceUrl + "scene-list-add", scn_list_add_payload, apiKey) 
count

58

In [22]:
sendRequest(serviceUrl + "scene-list-get", {'listId' : scn_list_add_payload['listId']}, apiKey)  

[{'entityId': 'LC81860572022262LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC81850582022255LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC91850582022247LGN01', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC81850582022239LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC91860572022238LGN01', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC81860572022166LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC91860572022158LGN01', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC91850572022151LGN01', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC81860572022150LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC81850582022143LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC91850572022135LGN01', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC91850582022135LGN01', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC81860572022134LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC91860572

Functions to support download 

In [23]:
def get_download_options(listId, datasetName, bandGroup):
    """
    Retrieve download options for a specified dataset.

    Parameters:
    - listId (str): The identifier for the list of items to download.
    - datasetName (str): The name of the dataset from which to obtain download options.
    - bandGroup (bool): A flag indicating whether to include secondary file groups. 
                        If True, secondary file groups will be included in the payload.

    Returns:
    - dict: A dictionary containing the available products for download.
    """
    
    # Prepare the payload for the download options request
    download_opt_payload = {
        "listId": listId,              
        "datasetName": datasetName      
    }

    # If bandGroup is specified, include the secondary file groups in the payload
    if bandGroup:
        download_opt_payload['includeSecondaryFileGroups'] = True

    # Print the payload for debugging purposes
    print(f"download_opt_payload: {download_opt_payload}")
    
    # Send request to the download options endpoint and retrieve list of available products
    products = sendRequest(serviceUrl + "download-options", download_opt_payload, apiKey)
    
    return products

In [24]:
def run_download_request(download_req_payload):
    """
    Sends a download request to the specified service and handles the response.

    Parameters:
    - download_req_payload (dict): The payload containing parameters needed to execute the download request. example: 
                                    {
                                    "downloads": [{'entityId': 'L2SR_LC08_L2SP_068018_20200310_20200822_02_T1_SR_B2_TIF',
                                                       'productId': '5f85f041a2ea6695'},
                                                      {'entityId': 'L2ST_LC08_L2SP_068018_20200310_20200822_02_T1_ST_B10_TIF',
                                                       'productId': '5f85f041a2ea6695'}],
                                    "label": '20250108_174449'
                                    } 
                                    where downloads is a list of entityIds and productIds for each Item being downloaded and a "label" is 
                                    a user define string 
    
    Returns:
    - dict: A dictionary of available URLs
    
    Exits the program if no records are returned from the download request.
    """

    print(f"Sending a download request...")
    
    # Send the download request using the provided payload and store the results
    download_request_results = sendRequest(serviceUrl + "download-request", download_req_payload, apiKey)

    # Check if any new records or duplicate products were returned
    if len(download_request_results['newRecords']) == 0 and len(download_request_results['duplicateProducts']) == 0:
        print('No records returned, please update your scenes or scene-search filter')
        sys.exit()
    else:
        return download_request_results

In [25]:
def run_download_retrieve(download_request_results, out_dir):
    
    # Attempt the download URLs
    for result in download_request_results['availableDownloads']:  
        # print(f"Get download url: {result['url']}\n" )
        runDownload(threads, result['url'], out_dir)
    
    # Get items labeled as being prepared for Download
    preparingDownloadCount = len(download_request_results['preparingDownloads'])
    preparingDownloadIds = []
    if preparingDownloadCount > 0:
        for result in download_request_results['preparingDownloads']:  
            preparingDownloadIds.append(result['downloadId'])

        download_ret_payload = {"label" : label}                
        # Retrieve download URLs
        print("Retrieving download urls...\n")
        download_retrieve_results = sendRequest(serviceUrl + "download-retrieve", download_ret_payload, apiKey, False)
        if download_retrieve_results != False:
            print(f"    Retrieved: \n" )
            for result in download_retrieve_results['available']:
                if result['downloadId'] in preparingDownloadIds:
                    preparingDownloadIds.remove(result['downloadId'])
                    runDownload(threads, result['url'], out_dir)
                    print(f"       {result['url']}\n" )

            for result in download_retrieve_results['requested']:   
                if result['downloadId'] in preparingDownloadIds:
                    preparingDownloadIds.remove(result['downloadId'])
                    runDownload(threads, result['url'], out_dir)
                    print(f"       {result['url']}\n" )

        # Didn't get all download URLs, retrieve again after 30 seconds
        while len(preparingDownloadIds) > 0: 
            print(f"{len(preparingDownloadIds)} downloads are not available yet. Waiting for 30s to retrieve again\n")
            time.sleep(30)
            download_retrieve_results = sendRequest(serviceUrl + "download-retrieve", download_ret_payload, apiKey, False)
            if download_retrieve_results != False:
                for result in download_retrieve_results['available']:                            
                    if result['downloadId'] in preparingDownloadIds:
                        preparingDownloadIds.remove(result['downloadId'])
                        print(f"    Get download url: {result['url']}\n" )
                        runDownload(threads, result['url'], out_dir)

    print(f"\nDownloading {len(download_request_results['availableDownloads'])} files... Please do not close the program\n")
    for thread in threads:
        thread.join() 

Select bands

In [26]:
bandNames = {'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6','SR_B7', 'ST_B10','QA_PIXEL', 'SR_QA_AEROSOL', 'QA_RADSAT', 'ANG'} 

Create download request

In [31]:
# Set your output directory
outDir = ("/Users/hstouter/Library/CloudStorage/Box-Box/NASA - Land Cover Land Use Change (2023 - 2026)/Data/LCLUC/RF_Datasets/landsat/LS8_9_python_test")
os.chdir(outDir)  # switch to that directory
print(outDir)

/Users/hstouter/Library/CloudStorage/Box-Box/NASA - Land Cover Land Use Change (2023 - 2026)/Data/LCLUC/RF_Datasets/landsat/LS8_9_python_test


In [35]:
products = get_download_options(listId, datasetName, False) 

print(products)

download_opt_payload: {'listId': 'temp_{datasetName}_list', 'datasetName': 'landsat_ot_c2_l2'}
[{'id': '5e83d14fec7cae84', 'downloadName': None, 'displayId': 'LC08_L2SP_185057_20211027_20211104_02_T1', 'entityId': 'LC81850572021300LGN00', 'datasetId': '5e83d14f2fc39685', 'available': True, 'filesize': 1065129625, 'productName': 'Landsat Collection 2 Level-2 Product Bundle', 'productCode': 'D694', 'bulkAvailable': True, 'downloadSystem': 'ls_zip', 'secondaryDownloads': [{'id': '5f85f041a2ea6695', 'downloadName': None, 'displayId': 'LC08_L2SP_185057_20211027_20211104_02_T1_ANG.txt', 'entityId': 'L2ST_LC08_L2SP_185057_20211027_20211104_02_T1_ANG_TXT', 'datasetId': '5f85f041c828327a', 'available': True, 'filesize': 117391, 'productName': 'Landsat Collection 2 Level-2 Band File', 'productCode': 'D693', 'bulkAvailable': True, 'downloadSystem': 'ls_s3', 'secondaryDownloads': [], 'fileGroups': None, 'displayOrder': 54, 'checksum': [{'id': 'sha512', 'value': 'f4e0a00f0f53f3d1568dd9437dd7cd75da0

In [36]:
pd.json_normalize(products) 

,id,downloadName,displayId,entityId,datasetId,available,filesize,productName,productCode,bulkAvailable,downloadSystem,secondaryDownloads,fileGroups
0,5e83d14fec7cae84,None,LC08_L2SP_185057_20211027_20211104_02_T1,LC81850572021300LGN00,5e83d14f2fc39685,True,1065129625,Landsat Collection 2 Level-2 Product Bundle,D694,True,ls_zip,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
1,6448198cc7b442a4,C2L2 Tile Product Files,LC08_L2SP_185057_20211027_20211104_02_T1,LC81850572021300LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D693,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
2,632210d4770592cf,None,LC08_L2SP_185057_20211027_20211104_02_T1,LC81850572021300LGN00,5e83d14f2fc39685,False,1065129625,Landsat Collection 2 Level-2 Product Bundle,D806,False,dds_ms,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
3,6448198c62023764,C2L2 Tile Product Files,LC08_L2SP_185057_20211027_20211104_02_T1,LC81850572021300LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D691,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
4,5e83d14fec7cae84,None,LC08_L2SP_185057_20211214_20211223_02_T1,LC81850572021348LGN00,5e83d14f2fc39685,True,1086176626,Landsat Collection 2 Level-2 Product Bundle,D694,True,ls_zip,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,6448198c62023764,C2L2 Tile Product Files,LC09_L2SP_186057_20220607_20230414_02_T1,LC91860572022158LGN01,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D691,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
228,5e83d14fec7cae84,None,LC09_L2SP_186057_20220826_20230401_02_T2,LC91860572022238LGN01,5e83d14f2fc39685,True,1021465592,Landsat Collection 2 Level-2 Product Bundle,D694,True,ls_zip,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
229,6448198cc7b442a4,C2L2 Tile Product Files,LC09_L2SP_186057_20220826_20230401_02_T2,LC91860572022238LGN01,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D693,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
230,632210d4770592cf,None,LC09_L2SP_186057_20220826_20230401_02_T2,LC91860572022238LGN01,5e83d14f2fc39685,False,1021465592,Landsat Collection 2 Level-2 Product Bundle,D806,False,dds_ms,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None


In [37]:
downloads = []
for product in products:  
    if product["secondaryDownloads"] is not None and len(product["secondaryDownloads"]) > 0:
        for secondaryDownload in product["secondaryDownloads"]:
            for bandName in bandNames:
                if secondaryDownload["bulkAvailable"] and bandName in secondaryDownload['displayId']:
                    downloads.append({"entityId":secondaryDownload["entityId"], "productId":secondaryDownload["id"]})

In [38]:
download_req_payload = {
        "downloads": downloads,
        "label": label
    }

download_req_payload

{'downloads': [{'entityId': 'L2ST_LC08_L2SP_185057_20211027_20211104_02_T1_ANG_TXT',
   'productId': '5f85f041a2ea6695'},
  {'entityId': 'L2SR_LC08_L2SP_185057_20211027_20211104_02_T1_QA_PIXEL_TIF',
   'productId': '5f85f041a2ea6695'},
  {'entityId': 'L2SR_LC08_L2SP_185057_20211027_20211104_02_T1_QA_RADSAT_TIF',
   'productId': '5f85f041a2ea6695'},
  {'entityId': 'L2SR_LC08_L2SP_185057_20211027_20211104_02_T1_SR_B1_TIF',
   'productId': '5f85f041a2ea6695'},
  {'entityId': 'L2SR_LC08_L2SP_185057_20211027_20211104_02_T1_SR_B2_TIF',
   'productId': '5f85f041a2ea6695'},
  {'entityId': 'L2SR_LC08_L2SP_185057_20211027_20211104_02_T1_SR_B3_TIF',
   'productId': '5f85f041a2ea6695'},
  {'entityId': 'L2SR_LC08_L2SP_185057_20211027_20211104_02_T1_SR_B4_TIF',
   'productId': '5f85f041a2ea6695'},
  {'entityId': 'L2SR_LC08_L2SP_185057_20211027_20211104_02_T1_SR_B5_TIF',
   'productId': '5f85f041a2ea6695'},
  {'entityId': 'L2SR_LC08_L2SP_185057_20211027_20211104_02_T1_SR_B6_TIF',
   'productId': '5f8

Send download request 

In [39]:
download_request_results = run_download_request(download_req_payload) 

Sending a download request...


Download request

In [44]:
run_download_retrieve(download_request_results, outDir) 



    Downloading: LC08_L2SP_185058_20220912_20220921_02_T1_QA_PIXEL.TIF -- https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2022/185/058/LC08_L2SP_185058_20220912_20220921_02_T1/LC08_L2SP_185058_20220912_20220921_02_T1_QA_PIXEL.TIF?requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6Mjc0MDcyNzQsImRvd25sb2FkSWQiOjgxODYzMTU0MCwiZGF0ZUdlbmVyYXRlZCI6IjIwMjUtMDYtMTFUMTI6MDY6MDktMDU6MDAiLCJpZCI6IkxDMDhfTDJTUF8xODUwNThfMjAyMjA5MTJfMjAyMjA5MjFfMDJfVDFfUUFfUElYRUwuVElGIiwic2lnbmF0dXJlIjoiJDUkJFBxb1ZCYjAuMlBqdU9vZjBkXC9mTFA2WGgyOHZSME5SWjAzXC9OVE5QSG1UNiJ9...
    Downloading: LC08_L2SP_185058_20220827_20220909_02_T2_SR_B7.TIF -- https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2022/185/058/LC08_L2SP_185058_20220827_20220909_02_T2/LC08_L2SP_185058_20220827_20220909_02_T2_SR_B7.TIF?requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6Mjc0MDcyNzQsImRvd25sb2FkSWQiOjgxODYzMTUzNiwiZGF0ZUdlbmVyYXRlZCI6IjIwMjUtMDYtMTFUMTI6MDY6MDgtMDU6MDAi